The spice kernels for TESS can be retrieved from [MAST](https://archive.stsci.edu/missions-and-data/tess/data-products).

In [60]:
# import os
import sys

sys.path.insert(0, r"/mnt/c/Users/Trez/Desktop/tudat-bundle/tudatpy/")
import pandas as pd
import numpy as np

# import astroquery.mpc.MPC as astroqueryMPC
from astroquery.mpc import MPC
import datetime
from typing import Union
import time

from tudatpy.kernel import constants
from tudatpy.kernel.interface import spice
from tudatpy.kernel import numerical_simulation
from tudatpy.kernel.numerical_simulation import environment_setup
from tudatpy.kernel.numerical_simulation import propagation_setup
from tudatpy.kernel.numerical_simulation import estimation, estimation_setup
from tudatpy.kernel.numerical_simulation.estimation_setup import observation
from tudatpy.kernel.astro import element_conversion

# from astropy.time import Time
from astropy.coordinates import EarthLocation
from astropy import units as u

# from pac.data import BatchMPC
from tudatpy.data.mpc import BatchMPC
from tudatpy.data.horizons import HorizonsQuery
# import tudatpy.data as data

import matplotlib.pyplot as plt

from acc import bods, allAccels



In [61]:
# SPICE KERNELS
spice.load_standard_kernels()
# spice.load_kernel(r"codes_300ast_20100725.bsp")
# spice.load_kernel(r"codes_300ast_20100725.tf")

# TESS - https://archive.stsci.edu/missions/tess/models/
# https://archive.stsci.edu/tess/bulk_downloads.html
# https://archive.stsci.edu/missions-and-data/tess/data-products.html#mod_eng

# 229 = Aug 17
# spice.load_kernel(r"TESS_EPH_DEF_2023229_21.bsp")
# spice.load_kernel(r"TESS_EPH_PRE_LONG_2023229_21.bsp")
# spice.load_kernel(r"tess_20_year_long_predictive.bsp")
# spice.load_kernel(r"TESS_EPH_PRE_LONG_2018319_01.bsp")
# spice.load_kernel(r"TESS_EPH_PRE_2021151_21.bsp")

# dont ask me how i know this
# TESS_naif = "-95"

In [62]:
codes = [433, 1]
codes = [433]
codes = [433, 1, 4]


start_dt = datetime.datetime(2018, 11, 1)
end_dt = datetime.datetime(2023, 6, 1)

start_dt_buff = datetime.datetime(2018, 6, 1)
end_dt_buff = datetime.datetime(2023, 8, 1)


batch = BatchMPC()
batch.get_observations(codes)
batch.filter(
    epoch_start=start_dt,
    epoch_end=end_dt,
    # observatories_exclude=["C51", "C59"],
    observatories_exclude=["C57"],
)

batch.summary()
# print(batch.table)
print(batch.observatories_table(only_in_batch=True, only_space_telescopes=True, include_positions=False))


   Batch Summary:
1. Batch includes 3 minor planets:
   ['433', '1', '4']
2. Batch includes 1746 observations, including 275 observations from space telescopes
3. The observations range from 2018-11-05 18:03:13.535985 to 2023-05-29 11:10:29.827215
   In seconds TDB since J2000: 594713062.7185509 to 738630699.0121946
   In Julian Days: 2458428.25224 to 2460093.965623
4. The batch contains observations from 65 observatories, including 2 space telescopes

     Code        Name  count
1225  C51        WISE  273.0
1232  C59  Yangwang-1    2.0


In [63]:
bodies_to_create = bods # + ["-95"]
central_bodies = ["Sun" for _ in batch.MPC_objects] #+ ["Earth"]
bodies_to_propagate = batch.MPC_objects# + ["TESS"]

In [64]:
wise_query = HorizonsQuery(
    query_id="WISE",
    location="@Sun",
    epoch_start=start_dt_buff,
    epoch_end=end_dt_buff,
    epoch_step="1h",
    extended_query=True
)
tess_query = HorizonsQuery(
    query_id="TESS",
    location="@Sun",
    epoch_start=start_dt_buff,
    epoch_end=end_dt_buff,
    epoch_step="1h",
    extended_query=True
)

In [65]:
global_frame_origin = "Sun"
global_frame_orientation = "J2000"
body_settings = environment_setup.get_default_body_settings(
    bodies_to_create, global_frame_origin, global_frame_orientation
)
# NOTE this way produces settings, the ephemeris itself is made by tudat
body_settings.add_empty_settings("TESS")
body_settings.get("TESS").ephemeris_settings = tess_query.create_ephemeris_tabulated(
    frame_origin=global_frame_origin,
    frame_orientation=global_frame_orientation,
    refplane="earth",
)
body_settings.add_empty_settings("WISE")
body_settings.get("WISE").ephemeris_settings = wise_query.create_ephemeris_tabulated(
    frame_origin=global_frame_origin,
    frame_orientation=global_frame_orientation,
    refplane="earth",
)

# Create system of bodies
bodies = environment_setup.create_system_of_bodies(body_settings)

# NOTE here you're trying set the ephemeris, which is not something made by the user and in this case is not made yet.
# NOTE tudat needs to make it itself so its consistent with the rest of the environment
# Add TESS
# bodies.create_empty_body("TESS")
# bodies.get("TESS").mass = 0
# bodies.get("TESS").ephemeris = environment_setup.ephemeris.direct_spice(
#      global_frame_origin, global_frame_orientation, TESS_naif)

satellites_to_include = {"C57": "TESS", "C51": "WISE"}
# satellites_to_include = {"C51": "WISE"}

In [66]:
observation_collection, links = batch.to_tudat(bodies=bodies,
                                               included_satellites=satellites_to_include
                                            )
observation_settings_list = list()
for link in list(links.values()):
    observation_settings_list.append(observation.angular_position(link))

In [67]:
end = batch.epoch_end
start = batch.epoch_start

In [68]:
acceleration_settings = {}
for body in batch.MPC_objects:
    acceleration_settings[str(body)] = allAccels

acceleration_settings_2 = acceleration_settings.copy()
# acceleration_settings["TESS"] = allAccels

acceleration_models = propagation_setup.create_acceleration_models(
    bodies, acceleration_settings, bodies_to_propagate, central_bodies
)
acceleration_models_2 = propagation_setup.create_acceleration_models(
    bodies, acceleration_settings_2, bodies_to_propagate, central_bodies
)

In [69]:
initial_state_Eros = spice.get_body_cartesian_state_at_epoch("Eros", "Sun", "J2000", "NONE", end)

initial_state_Ceres = spice.get_body_cartesian_state_at_epoch("Ceres", "Sun", "J2000", "NONE", end)

initial_state_Vesta = spice.get_body_cartesian_state_at_epoch("Vesta", "Sun", "J2000", "NONE", end)
initial_states = np.concatenate([initial_state_Eros, initial_state_Ceres, initial_state_Vesta])

# Add random initial states
initial_states[0:3] += + np.random.rand(3)*1000e3
initial_states[6:9] += + np.random.rand(3)*1000e3
initial_states[12:15] += + np.random.rand(3)*1000e3

initial_states[3:6] += + np.random.rand(3)*10
initial_states[9:12] += + np.random.rand(3)*10
initial_states[15:18] += + np.random.rand(3)*10


In [70]:
termination_condition = propagation_setup.propagator.time_termination(start)


dt = -60000
# Create numerical integrator settings
integrator_settings = propagation_setup.integrator.runge_kutta_variable_step_size(
    end, dt, propagation_setup.integrator.rkf_78, dt, dt, 1.0, 1.0
)

# Create propagation settings
propagator_settings = propagation_setup.propagator.translational(
    central_bodies=central_bodies,
    acceleration_models=acceleration_models,
    bodies_to_integrate=bodies_to_propagate,
    initial_states=initial_states,
    initial_time=end,
    integrator_settings=integrator_settings,
    termination_settings=termination_condition,
)
propagator_settings_2 = propagation_setup.propagator.translational(
    central_bodies=central_bodies[0:1],
    acceleration_models=acceleration_models_2,
    bodies_to_integrate=bodies_to_propagate[0:1],
    initial_states=initial_states[0:6],
    initial_time=end,
    integrator_settings=integrator_settings,
    termination_settings=termination_condition,
)

In [71]:
# Setup parameters settings to propagate the state transition matrix
parameter_settings = estimation_setup.parameter.initial_states(
    propagator_settings_2, bodies
)

# Create the parameters that will be estimated
parameters_to_estimate = estimation_setup.create_parameter_set(
    parameter_settings, bodies, propagator_settings
)

print(central_bodies)
print(bodies_to_propagate)

['Sun', 'Sun', 'Sun']
['433', '1', '4']


In [72]:

estimator = numerical_simulation.Estimator(
    bodies=bodies,
    estimated_parameters=parameters_to_estimate,
    observation_settings=observation_settings_list,
    propagator_settings=propagator_settings_2,
    integrate_on_creation=True, 
)


pod_input = estimation.EstimationInput(
    observations_and_times=observation_collection,
    convergence_checker=estimation.estimation_convergence_checker(
        maximum_iterations=3,
    )
)

RuntimeError: Error, provided acceleration models for body 1, but this body is not included in list of bodies for which translational dynamics is to be propagated.

In [ ]:
pod_input.define_estimation_settings(reintegrate_variational_equations=True)


In [ ]:
pod_output = estimator.perform_estimation(pod_input)
results = parameters_to_estimate.parameter_vector
results = pod_output.parameter_history[:, -1]

Calculating residuals and partials 2568
Parameter update   -78853.8     25952.7    -21192.2  0.00250479 -0.00110328  0.00588809
Current residual: 9.34856e-06
Calculating residuals and partials 2568
Parameter update  -0.0503696   -0.0643886    -0.339971 -1.14611e-08 -1.28092e-08 -1.36402e-08
Current residual: 9.24994e-06
Calculating residuals and partials 2568
Parameter update  0.00162546   0.00124746 -0.000229854 -1.81431e-10 -4.13195e-11  1.97589e-10
Current residual: 9.24994e-06
Maximum number of iterations reached
Final residual: 9.24994e-06


In [ ]:
result_eros = results[0:6]
result_ceres = results[6:12]
result_vesta = results[12:18]



In [ ]:
print(f"Eros radial error: {np.sqrt(np.square((np.array(result_eros) - initial_state_Eros)[0:3]).sum())/1000} km")
print(f"Ceres radial error: {np.sqrt(np.square((np.array(result_ceres) - initial_state_Ceres)[0:3]).sum())/1000} km")
print(f"Vesta radial error: {np.sqrt(np.square((np.array(result_vesta) - initial_state_Vesta)[0:3]).sum())/1000} km")


Eros radial error: 85.67729476507829 km


NameError: name 'initial_state_Ceres' is not defined